In [6]:
from data_preparation_functions import *
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import warnings
from sklearn import linear_model, tree, discriminant_analysis, naive_bayes, ensemble, gaussian_process
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.metrics import log_loss, confusion_matrix
import xgboost as xgb
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)

# EPL Machine Learning Walkthrough

## 03. Model Building & Hyperparameter Tuning
Welcome to the third part of this Machine Learning Walkthrough. This tutorial will focus on the model building process, including how to tune hyperparameters. In the [next tutorial], we will create weekly predictions based on the model we have created here.

Specifically, this tutorial will cover a few things:

1. Choosing which Machine Learning algorithm to use from a variety of choices
2. Hyperparameter Tuning
3. Overfitting/Underfitting

### Choosing an Algorithm
The best way to decide on specific algorithm to use, is to try them all! To do this, we will define a function which we first used in our AFL Predictions tutorial. This will iterate over a number of algorithms and give us a good indication of which algorithms are suited for this dataset and exercise.

Let's first use grab the features we created in the last tutorial. This may take a minute or two to run.

In [7]:
features = create_feature_df()

Creating all games feature DataFrame
Creating stats feature DataFrame
Creating odds feature DataFrame
Creating market values feature DataFrame
Filling NAs
Merging stats, odds and market values into one features DataFrame
Complete.


To start our modelling process, we need to make a training set, a test set and a holdout set. As we are using cross validation, we will make our training set all of the seasons up until 2018/19, and we will use the 2018/19 season as the test set.

In [18]:
feature_list = [col for col in features.columns if col.startswith("f_")]
betting_features = []

le = LabelEncoder() # Initiate a label encoder to transform the labels 'away', 'draw', 'home' to 0, 1, 2

# Grab all seasons except for 18/19 to use CV with
all_x = features.loc[features.season != '1819', ['gameId'] + feature_list]
all_y = features.loc[features.season != '1819', 'result']
all_y = le.fit_transform(all_y)

# Create our training vector as the seasons except 17/18 and 18/19
train_x = features.loc[~features.season.isin(['1718', '1819']), ['gameId'] + feature_list]
train_y = le.transform(features.loc[~features.season.isin(['1718', '1819']), 'result'])

# Create our holdout vectors as the 17/18 season
holdout_x = features.loc[features.season == '1718', ['gameId'] + feature_list]
holdout_y = le.transform(features.loc[features.season == '1718', 'result'])

# Create our test vectors as the 18/19 season
test_x = features.loc[features.season == '1819', ['gameId'] + feature_list]
test_y = le.transform(features.loc[features.season == '1819', 'result'])

In [5]:
import h2o

h2o.init(nthreads = -1, max_mem_size = 8)
h2o.connect()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_152-release"; OpenJDK Runtime Environment (build 1.8.0_152-release-1056-b12); OpenJDK 64-Bit Server VM (build 25.152-b12, mixed mode)
  Starting server from /home/groninge/anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpa5g2821u
  JVM stdout: /tmp/tmpa5g2821u/h2o_groninge_started_from_python.out
  JVM stderr: /tmp/tmpa5g2821u/h2o_groninge_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Europe/Amsterdam
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.5
H2O cluster version age:,4 days
H2O cluster name:,H2O_from_python_groninge_u1gi8k
H2O cluster total nodes:,1
H2O cluster free memory:,7.111 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


Connecting to H2O server at http://localhost:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Europe/Amsterdam
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.5
H2O cluster version age:,4 days
H2O cluster name:,H2O_from_python_groninge_u1gi8k
H2O cluster total nodes:,1
H2O cluster free memory:,7.111 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


<H2OConnection to http://localhost:54321, no session>

In [60]:
train = features.loc[~features.season.isin(['1718', '1819']), ['result'] + feature_list]
valid = features.loc[features.season == '1718', ['result'] + feature_list]
test = features.loc[features.season == '1819', ['result'] + feature_list]

train.to_csv("data/train.csv", index=False)
valid.to_csv("data/valid.csv", index=False)
test.to_csv("data/test.csv", index=False)

train_h2o = h2o.import_file("data/train.csv")
valid_h2o = h2o.import_file("data/valid.csv")
test_h2o = h2o.import_file("data/test.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [65]:
from h2o.automl import H2OAutoML

aml = H2OAutoML(
    max_runtime_secs = 60 * 30,
    max_models = 100,
    stopping_metric = "logloss",
    sort_metric = "logloss",
    balance_classes = True,
    seed = 183)

aml.train(y='result', training_frame=train_h2o, validation_frame=valid_h2o)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [66]:
lb = aml.leaderboard
lb.head()

model_id,mean_per_class_error,logloss,rmse,mse
GLM_grid_1_AutoML_20190920_224105_model_1,0.550189,0.96451,0.605731,0.36691
GLM_grid_1_AutoML_20190920_223735_model_1,0.550189,0.96451,0.605731,0.36691
StackedEnsemble_BestOfFamily_AutoML_20190920_224105,0.545782,0.965398,0.605491,0.366619
StackedEnsemble_AllModels_AutoML_20190920_223735,0.549286,0.965968,0.605848,0.367052
StackedEnsemble_BestOfFamily_AutoML_20190920_223735,0.54923,0.966075,0.605942,0.367166
GBM_grid_1_AutoML_20190920_224105_model_16,0.548762,0.96982,0.607167,0.368652
XGBoost_3_AutoML_20190920_224105,0.548354,0.972219,0.607742,0.369351
XGBoost_3_AutoML_20190920_223735,0.548354,0.972219,0.607742,0.369351
XGBoost_grid_1_AutoML_20190920_224105_model_4,0.551821,0.973544,0.608221,0.369932
GBM_grid_1_AutoML_20190920_224105_model_21,0.551776,0.974303,0.609266,0.371205


In [67]:
aml.leader.hit_ratio_table(valid=True)


Top-3 Hit Ratios: 

,k,hit_ratio
0,1,0.545213
1,2,0.789894
2,3,1.000000


In [68]:
aml.leader.confusion_matrix(valid_h2o)


Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,away,draw,home,Error,Rate
0,55.0,0.0,51.0,0.481132,51 / 106
1,22.0,0.0,77.0,1.000000,99 / 99
2,21.0,0.0,150.0,0.122807,21 / 171
3,98.0,0.0,278.0,0.454787,171 / 376


In [69]:
# save the model
model_path = h2o.save_model(model=aml.leader, path="data/mymodel", force=True)

print(model_path)

/home/groninge/projects/predictive-models/epl/data/mymodel/GLM_grid_1_AutoML_20190920_224105_model_1


In [71]:
h2o.cluster().shutdown()

In [5]:
# Create a list of standard classifiers
classifiers = [

    #GLM
    linear_model.LogisticRegressionCV(),
    
    #Navies Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),
    
    #Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),

    #Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #Gaussian Processes
    gaussian_process.GaussianProcessClassifier(),
    
    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
    xgb.XGBClassifier()    
]

In [6]:
def find_best_algorithms(classifier_list, X, y):
    # This function is adapted from https://www.kaggle.com/yassineghouzam/titanic-top-4-with-ensemble-modeling
    # Cross validate model with Kfold stratified cross validation
    kfold = StratifiedKFold(n_splits=5)
    
    # Grab the cross validation scores for each algorithm
    cv_results = [cross_val_score(classifier, X, y, scoring = "neg_log_loss", cv = kfold) for classifier in classifier_list]
    cv_means = [cv_result.mean() * -1 for cv_result in cv_results]
    cv_std = [cv_result.std() for cv_result in cv_results]
    algorithm_names = [alg.__class__.__name__ for alg in classifiers]
    
    # Create a DataFrame of all the CV results
    cv_results = pd.DataFrame({
        "Mean Log Loss": cv_means,
        "Log Loss Std": cv_std,
        "Algorithm": algorithm_names
    }).sort_values(by='Mean Log Loss')
    return cv_results

In [7]:
algorithm_results = find_best_algorithms(classifiers, all_x, all_y)

In [8]:
algorithm_results

,Mean Log Loss,Log Loss Std,Algorithm
0,0.966852,0.008842,LogisticRegressionCV
1,1.016643,0.009693,BernoulliNB
3,1.046766,0.101457,LinearDiscriminantAnalysis
5,1.093913,0.008338,AdaBoostClassifier
11,1.096977,0.139928,XGBClassifier
10,1.098612,0.000000,GaussianProcessClassifier
8,1.105117,0.089326,GradientBoostingClassifier
9,2.111492,0.201660,RandomForestClassifier
7,2.112677,0.173546,ExtraTreesClassifier
6,2.453923,0.274825,BaggingClassifier


We can see that LogisticRegression seems to perform the best out of all the algorithms, and some algorithms have a very high log loss. This is most likely due to overfitting. It would definitely be useful to condense our features down to reduce the dimensionality of the dataset.

### Hyperparameter Tuning
For now, however, we will use logistic regression. Let's first try and tune a logistic regression model with cross validation. To do this, we will use [grid search](https://en.wikipedia.org/wiki/Hyperparameter_optimization). Grid search essentially tries out each combination of values and finds the model with the lowest error metric, which in our case is log loss. 'C' in logistic regression determines the amount of regularization. Lower values increase regularization.

In [9]:
# Define our parameters to run a grid search over
lr_grid = {
    "C": [0.0001, 0.01, 0.05, 0.2, 1],
    "solver": ["newton-cg", "lbfgs", "liblinear"]
}

kfold = StratifiedKFold(n_splits=5)

gs = GridSearchCV(LogisticRegression(), param_grid=lr_grid, cv=kfold, scoring='neg_log_loss')
gs.fit(all_x, all_y)
print("Best log loss: {}".format(gs.best_score_ *-1))
best_lr_params = gs.best_params_
print("Best lr params: {}".format(best_lr_params))

Best log loss: 0.9677573033266084
Best lr params: {'C': 0.05, 'solver': 'lbfgs'}


### Defining a Baseline
We should also define a baseline, as we don't really know if our log loss is good or bad. Randomly assigning a 1/3 chance to each selection yields a log loss of log3 = 1.09. However, what we are really interested in, is how our model performs relative to the odds. So let's find the log loss of the odds.

In [9]:
# Finding the log loss of the odds
log_loss(all_y, 1 / all_x[['f_awayOdds', 'f_drawOdds', 'f_homeOdds']])

0.9591793301858665

This is good news; our algorithm almost beats the bookies in terms of log loss. It would be great if we could beat this result.

### Analysing the Errors Made
Now that we have a logistic regression model tuned, let's see what type of errors it made. To do this we will look at the confusion matrix produced when we predict our holdout set.

In [10]:
lr = LogisticRegression(**best_lr_params) # Instantiate the model
lr.fit(train_x, train_y) # Fit our model
lr_predict = lr.predict(holdout_x) # Predict the holdout values

In [11]:
# Create a confusion matrix
c_matrix = (pd.DataFrame(confusion_matrix(holdout_y, lr_predict), columns=le.classes_, index=le.classes_)
 .rename_axis('Actual')
 .rename_axis('Predicted', axis='columns'))

c_matrix

Predicted,away,draw,home
Actual,,,
away,58,0,48
draw,25,0,74
home,22,0,149


As we can see, when we predicted 'away' as the result, we correctly predicted 79 / 109 results, a hit rate of 70.6%. However, when we look at our draw hit rate, we only predicted 6 / 84 correctly, meaning we only had a hit rate of around 8.3%. For a more in depth analysis of our predictions, please skip to the Analysing Predictions & Staking Strategies section of the tutorial.

Before we move on, however, let's use our model to predict the 17/18 season and compare how we went with the odds.

In [14]:
FP = c_matrix.sum(axis=0) - np.diag(c_matrix)  
FN = c_matrix.sum(axis=1) - np.diag(c_matrix)
TP = np.diag(c_matrix)
TN = c_matrix.values.sum() - (FP + FN + TP)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)

# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)

print("Sensitivity, hit rate, recall, or true positive rate: {}\n".format(TPR))
print("Specificity or true negative rate {}\n".format(TNR))
print("Precision or positive predictive value: {}\n".format(PPV))
print("Negative predictive value: {}\n".format(NPV))
print("Fall out or false positive rate: {}\n".format(FPR))
print("False negative rate: {}\n".format(FNR))
print("False discovery rate: {}\n".format(FDR))
print("Overall accuracy: {}".format(ACC))

Sensitivity, hit rate, recall, or true positive rate: Actual
away    0.547170
draw    0.000000
home    0.871345
dtype: float64

Specificity or true negative rate Predicted
away    0.825926
draw    1.000000
home    0.404878
dtype: float64

Precision or positive predictive value: Predicted
away    0.552381
draw         NaN
home    0.549815
dtype: float64

Negative predictive value: Predicted
away    0.822878
draw    0.736702
home    0.790476
dtype: float64

Fall out or false positive rate: Predicted
away    0.174074
draw    0.000000
home    0.595122
dtype: float64

False negative rate: Actual
away    0.452830
draw    1.000000
home    0.128655
dtype: float64

False discovery rate: Predicted
away    0.447619
draw         NaN
home    0.450185
dtype: float64

Overall accuracy: Predicted
away    0.747340
draw    0.736702
home    0.617021
dtype: float64


In [12]:
# Get test predictions

test_lr = LogisticRegression(**best_lr_params)
test_lr.fit(all_x, all_y)
test_predictions_probs = lr.predict_proba(test_x)
test_predictions = lr.predict(test_x)

test_ll = log_loss(test_y, test_predictions_probs)
test_accuracy = (test_predictions == test_y).mean()

print("Our predictions for the 2018/19 season have a log loss of: {0:.5f} and an accuracy of: {1:.2f}".format(test_ll, test_accuracy))

Our predictions for the 2018/19 season have a log loss of: 0.89451 and an accuracy of: 0.59


In [13]:
# Get accuracy and log loss based on the odds
odds_ll = log_loss(test_y, 1 / test_x[['f_awayOdds', 'f_drawOdds', 'f_homeOdds']])

odds_predictions = test_x[['f_awayOdds', 'f_drawOdds', 'f_homeOdds']].apply(lambda row: row.idxmin()[2:6], axis=1).values
odds_accuracy = (odds_predictions == le.inverse_transform(test_y)).mean()

print("Odds predictions for the 2018/19 season have a log loss of: {0:.5f} and an accuracy of: {1:.3f}".format(odds_ll, odds_accuracy))

Odds predictions for the 2018/19 season have a log loss of: 0.89290 and an accuracy of: 0.584


### Results

There we have it! The odds predicted 60% of EPL games correctly in the 2018/19 season, whilst our model predicted 60% correctly. This is a decent result for the first iteration of our model. In future iterations, we could wait a certain number of matches each season and calculate EMAs for on those first n games. This may help the issue of players switching clubs and teams becoming relatively stronger/weaker compared to previous seasons.